![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/openvino/HuggingFace_OpenVINO_in_Spark_NLP_CamemBertForQuestionAnswering.ipynb)

# Import OpenVINO CamemBertForQuestionAnswering models from HuggingFace 🤗 into Spark NLP 🚀

This notebook provides a detailed walkthrough on optimizing and exporting CamemBertForQuestionAnswering models from HuggingFace for use in Spark NLP, leveraging the various tools provided in the [Intel OpenVINO toolkit](https://www.intel.com/content/www/us/en/developer/tools/openvino-toolkit/overview.html) ecosystem.

Let's keep in mind a few things before we start 😊

- OpenVINO support was introduced in  `Spark NLP 5.4.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for CamemBertForQuestionAnswering from CamemBertForQuestionAnswering and they have to be in `Question Answering` category.

## 1. Export and Save the HuggingFace model

- Let's install `transformers` and `openvino` packages with other dependencies. You don't need `openvino` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [ ]:
!pip install -q --upgrade transformers==4.48.3 optimum[openvino]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.6/342.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 862.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.5 MB/s eta 0:00:00

[Optimum Intel](https://github.com/huggingface/optimum-intel?tab=readme-ov-file#openvino) is the interface between the Transformers library and the various model optimization and acceleration tools provided by Intel. HuggingFace models loaded with optimum-intel are automatically optimized for OpenVINO, while being compatible with the Transformers API.
- To load a HuggingFace model directly for inference/export, just replace the `AutoModelForXxx` class with the corresponding `OVModelForXxx` class. We can use this to import and export OpenVINO models with `from_pretrained` and `save_pretrained`.
- By setting `export=True`, the source model is converted to OpenVINO IR format on the fly.
- We'll use [etalab-ia/camembert-base-squadFR-fquad-piaf](https://huggingface.co/etalab-ia/camembert-base-squadFR-fquad-piaf) model from HuggingFace as an example and load it as a `OVModelForQuestionAnswering`, representing an OpenVINO model.
- In addition to the OVModelForQuestionAnswering model, we also need to save the `AutoTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [ ]:
from optimum.intel import OVModelForQuestionAnswering
from transformers import AutoTokenizer

MODEL_NAME = "etalab-ia/camembert-base-squadFR-fquad-piaf"
EXPORT_PATH = f"ov_models/{MODEL_NAME}"

ov_model = OVModelForQuestionAnswering.from_pretrained(MODEL_NAME, export=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

ov_model.save_pretrained(EXPORT_PATH)
tokenizer.save_pretrained(EXPORT_PATH)

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


('ov_models/etalab-ia/camembert-base-squadFR-fquad-piaf/tokenizer_config.json',
 'ov_models/etalab-ia/camembert-base-squadFR-fquad-piaf/special_tokens_map.json',
 'ov_models/etalab-ia/camembert-base-squadFR-fquad-piaf/sentencepiece.bpe.model',
 'ov_models/etalab-ia/camembert-base-squadFR-fquad-piaf/added_tokens.json',
 'ov_models/etalab-ia/camembert-base-squadFR-fquad-piaf/tokenizer.json')

- Let's have a look inside these two directories and see what we are dealing with:

In [ ]:
!ls -l {EXPORT_PATH}

total 433368
-rw-r--r-- 1 root root       709 Jun 20 02:34 config.json
-rw-r--r-- 1 root root 440135888 Jun 20 02:34 openvino_model.bin
-rw-r--r-- 1 root root    377495 Jun 20 02:34 openvino_model.xml
-rw-r--r-- 1 root root    810912 Jun 20 02:34 sentencepiece.bpe.model
-rw-r--r-- 1 root root       354 Jun 20 02:34 special_tokens_map.json
-rw-r--r-- 1 root root      1674 Jun 20 02:34 tokenizer_config.json
-rw-r--r-- 1 root root   2421069 Jun 20 02:34 tokenizer.json


- We need to move the `sentencepiece.bpe.model` file from the tokenizer into an assets folder, as this is where Spark NLP looks for it when working with models like Camembert or other SentencePiece-based tokenizers.

In [ ]:
!mkdir -p {EXPORT_PATH}/assets & mv {EXPORT_PATH}/sentencepiece.bpe.model {EXPORT_PATH}/assets/

In [ ]:
!ls -l {EXPORT_PATH}/assets

total 792
-rw-r--r-- 1 root root 810912 Jun 20 02:34 sentencepiece.bpe.model


- Voila! We have our `sentencepiece.bpe.model` inside assets directory

## Import and Save CamemBertForQuestionAnswering in Spark NLP


- Let's install and setup Spark NLP in Google Colab.
- For this example, we'll use specific versions of `pyspark` and `spark-nlp` that we've already tested with this transformer model to make sure everything runs smoothly.

In [ ]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 27.0 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [ ]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: {}".format(sparknlp.version()))
print("Apache Spark version: {}".format(spark.version))

Spark NLP version: 5.5.3
Apache Spark version: 3.5.4


- Let's use `loadSavedModel` functon in `CamemBertForQuestionAnswering` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `CamemBertForQuestionAnswering` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.


In [ ]:
from sparknlp.annotator import CamemBertForQuestionAnswering

spanClassifier = CamemBertForQuestionAnswering.loadSavedModel(
     f"{EXPORT_PATH}",
     spark
 )\
  .setInputCols(["document_question",'document_context'])\
  .setOutputCol("answer")\
  .setCaseSensitive(False)\
  .setMaxSentenceLength(512)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [ ]:
spanClassifier.write().overwrite().save("./{}_spark_nlp_openvino".format(EXPORT_PATH))

Let's clean up stuff we don't need anymore

In [ ]:
!rm -rf {EXPORT_PATH}

Awesome 😎  !

This is your CamemBertForQuestionAnswering model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [ ]:
! ls -l {EXPORT_PATH}_spark_nlp_openvino

total 431056
-rw-r--r-- 1 root root 440580881 Jun 20 02:37 camembert_classification_openvino
-rw-r--r-- 1 root root    810912 Jun 20 02:37 camembert_spp
drwxr-xr-x 2 root root      4096 Jun 20 02:37 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny CamemBertForQuestionAnswering model in Spark NLP 🚀 pipeline!

In [ ]:
from sparknlp.base import MultiDocumentAssembler
from sparknlp.annotator import CamemBertForQuestionAnswering
from pyspark.ml import Pipeline

document_assembler = MultiDocumentAssembler() \
    .setInputCols(["question", "context"]) \
    .setOutputCols(["document_question", "document_context"])

spanClassifier_loaded = CamemBertForQuestionAnswering.load(f"./{EXPORT_PATH}_spark_nlp_openvino") \
    .setInputCols(["document_question", "document_context"]) \
    .setOutputCol("answer")

pipeline = Pipeline(stages=[
    document_assembler,
    spanClassifier_loaded
])

example = spark.createDataFrame([
    ["Où est-ce que je vis?", "Mon nom est Wolfgang et je vis à Berlin"]
]).toDF("question", "context")

model = pipeline.fit(example)
result = model.transform(example)

result.select("question", "context", "answer.result").show(truncate=False)

+---------------------+---------------------------------------+--------+
|question             |context                                |result  |
+---------------------+---------------------------------------+--------+
|Où est-ce que je vis?|Mon nom est Wolfgang et je vis à Berlin|[berlin]|
+---------------------+---------------------------------------+--------+



That's it! You can now go wild and use hundreds of `CamemBertForQuestionAnswering` models from HuggingFace 🤗 in Spark NLP 🚀
